In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import cPickle

torch.manual_seed(1)

**Prepare data:**

In [3]:
# def prepare_sequence(seq, to_ix):
#     idxs = [to_ix[w] for w in seq]
#     tensor = torch.LongTensor(idxs)
#     return autograd.Variable(tensor)

def prepare_sequence(idxs):
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)


target_dir = '/mnt/d/data/tagger/dataset/'
word_to_ix, tag_to_ix, sents_idx, labels_idx = cPickle.load(open(target_dir + "CoNLL_test.pkl", "rb"))

# training_data = [
#     ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
#     ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
# ]
# word_to_ix = {}
# for sent, tags in training_data:
#     for word in sent:
#         if word not in word_to_ix:
#             word_to_ix[word] = len(word_to_ix)
# print(word_to_ix)
# tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

print(word_to_ix['I'])
print(labels_idx[0])

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 64
HIDDEN_DIM = 32

483
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]


**Create the model:**

In [4]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
                autograd.Variable(torch.zeros(1, 1, self.hidden_dim)))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [6]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))

before training

In [7]:
# inputs = prepare_sequence(training_data[0][0], word_to_ix)
inputs = prepare_sequence(sents_idx[0])
tag_scores = model(inputs)
_, pred_tag = torch.max(tag_scores.data, 1)
print(pred_tag)


 3
 5
 1
 1
 3
 1
 1
 5
 3
 3
 5
[torch.LongTensor of size 11]



**Train the model:**

In [8]:
loss_function = nn.NLLLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.1)
optimizer = optim.RMSprop(model.parameters())


EPOCHS = 20
for epoch in range(EPOCHS):  # again, normally you would NOT do 300 epochs, it is toy data
#     for sentence, tags in training_data:

    print "epoch",  epoch + 1 , "/" , EPOCHS
    
    loss = 0

    for i, (sentence, tags) in enumerate(zip(sents_idx, labels_idx)):
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        model.hidden = model.init_hidden()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Variables of word indices.
        sentence_in = prepare_sequence(sentence)
        targets = prepare_sequence(tags)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        
        loss += loss.data[0]
        
    print 'loss: %.4f' % loss


epoch 1 / 20
loss: 1.9245
epoch 2 / 20
loss: 1.3236
epoch 3 / 20
loss: 1.9969
epoch 4 / 20
loss: 0.8087
epoch 5 / 20
loss: 0.2894
epoch 6 / 20
loss: 0.3681
epoch 7 / 20
loss: 0.1101
epoch 8 / 20
loss: 0.2340
epoch 9 / 20
loss: 0.1501
epoch 10 / 20
loss: 0.2665
epoch 11 / 20
loss: 0.0144
epoch 12 / 20
loss: 0.0112
epoch 13 / 20
loss: 0.0107
epoch 14 / 20
loss: 0.0081
epoch 15 / 20
loss: 0.0048
epoch 16 / 20
loss: 0.0083
epoch 17 / 20
loss: 0.0038
epoch 18 / 20
loss: 0.0041
epoch 19 / 20
loss: 0.0015
epoch 20 / 20
loss: 0.0009


after training

In [9]:
# inputs = prepare_sequence(training_data[0][0], word_to_ix)
inputs = prepare_sequence(sents_idx[0])
tag_scores = model(inputs)
_, pred_tag = torch.max(tag_scores.data, 1)
print(pred_tag)


 0
 0
 1
 0
 0
 0
 0
 0
 0
 0
 0
[torch.LongTensor of size 11]

